In [1]:
import pandas as pd
import numpy as np
from datetime import datetime

# Load the data
data = pd.read_csv('continuous dataset.csv')

# Convert timestamp to datetime format
data['datetime'] = data['datetime'][:45261].apply(lambda x: datetime.strptime(x, '%d-%m-%Y %H:%M'))

# Set the datetime as index
data.set_index('datetime', inplace=True)

# Resample the data to hourly frequency
data = data.resample('H').mean()

# Create lag features
for i in range(1, 25):
    data['lag_{}'.format(i)] = data['nat_demand'].shift(i)

# Create rolling mean and standard deviation features
data['rolling_mean'] = data['nat_demand'].rolling(window=24).mean()
data['rolling_std'] = data['nat_demand'].rolling(window=24).std()

# Create weekday and hour features
data['weekday'] = data.index.weekday
data['hour'] = data.index.hour

# Remove missing values
data.dropna(inplace=True)

# Split the data into train and test sets
train_size = int(len(data) * 0.8)
train_data = data[:train_size]/1000
test_data = data[train_size:]/1000

In [3]:
#Attention
import tensorflow as tf
from tensorflow.keras.layers import Input, Dense, LSTM, Conv1D, MaxPooling1D, Flatten, Dropout, Attention
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
from keras.losses import mean_squared_error


# Prepare the train and test data
X_train, y_train = train_data.drop('nat_demand', axis=1), train_data['nat_demand']
X_test, y_test = test_data.drop('nat_demand', axis=1), test_data['nat_demand']

# Reshape the data 
X_train = X_train.values.reshape(X_train.shape[0], X_train.shape[1], 1)
X_test = X_test.values.reshape(X_test.shape[0], X_test.shape[1], 1)

# Define the input shape
input_shape = (X_train.shape[1], 1)

# Define the model
inputs = Input(shape=input_shape)
conv1 = Conv1D(filters=64, kernel_size=3, activation='relu')(inputs)
pool1 = MaxPooling1D(pool_size=2)(conv1)
lstm1 = LSTM(64, return_sequences=True)(pool1)
attention = Attention()([lstm1, lstm1])
flatten = Flatten()(attention)
dropout = Dropout(0.2)(flatten)
outputs = Dense(1)(dropout)
model = Model(inputs=inputs, outputs=outputs)

# Compile the model
model.compile(loss='mse', optimizer=Adam())

# Train the model
history = model.fit(X_train, y_train, epochs=50, batch_size=64, validation_data=(X_test, y_test))

# Evaluate the model
mae = history.history['val_loss'][-1]
mse = mean_squared_error(y_test, model.predict(X_test))
rmse = np.sqrt(mse)

ypred = model.predict(X_test)

Epoch 1/50
566/566 [==============================] - 12s 18ms/step - loss: 0.0291 - val_loss: 0.0031
Epoch 2/50
566/566 [==============================] - 10s 17ms/step - loss: 0.0044 - val_loss: 0.0023
Epoch 3/50
566/566 [==============================] - 10s 18ms/step - loss: 0.0036 - val_loss: 0.0021
Epoch 4/50
566/566 [==============================] - 10s 18ms/step - loss: 0.0029 - val_loss: 0.0027
Epoch 5/50
566/566 [==============================] - 10s 18ms/step - loss: 0.0026 - val_loss: 0.0016
Epoch 6/50
566/566 [==============================] - 11s 19ms/step - loss: 0.0023 - val_loss: 0.0016
Epoch 7/50
566/566 [==============================] - 10s 18ms/step - loss: 0.0021 - val_loss: 0.0015
Epoch 8/50
566/566 [==============================] - 10s 18ms/step - loss: 0.0021 - val_loss: 0.0014
Epoch 9/50
566/566 [==============================] - 11s 19ms/step - loss: 0.0019 - val_loss: 0.0016
Epoch 10/50
566/566 [==============================] - 10s 18ms/step - loss: 0.001

In [4]:
#For MAE
mae = np.mean(np.abs(ypred.ravel() - y_test.ravel()))
print('MAE = {:.2f} (%)'.format(mae))

#For MAPE
mape = np.mean(np.abs((y_test.ravel() - ypred.ravel()) / y_test.ravel())) * 100
print('MAPE = {:.2f} (%)'.format(mape))


# For RMSE
from sklearn.metrics import mean_squared_error 
mse=mean_squared_error(y_test,ypred) 
rsme=np.sqrt(mse) 
print('RSME = {:.2f} (%)'.format(rsme))

MAE = 0.02 (%)
MAPE = 1.84 (%)
RSME = 0.03 (%)
